# Tarea 4 - Miguel Rios Tangarife - Punto 2

In [1]:
import numpy as np
import os
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import tarfile
import zipfile
from  sklearn.linear_model import LogisticRegression
from sklearn.cluster import AgglomerativeClustering
from scipy.io import loadmat
from urllib.request import urlretrieve
from os.path import isfile, isdir
import plotly.figure_factory as ff
import plotly.express as px
import statsmodels.api as sm
from IPython.display import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.datasets import imdb
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import glob
from sklearn.decomposition import PCA
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize.toktok import ToktokTokenizer
from bs4 import BeautifulSoup
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from keras import regularizers
from keras.models import Sequential
from keras import layers
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l2
from keras.layers import Embedding, LSTM, Dense
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to C:\Users\Miguel
[nltk_data]     Rios\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Regresión logistica

In [2]:
# Cargar los datos
dataTrain = pd.read_csv('data/train/trainingpunto2.csv')
dataTest = pd.read_csv('data/test/testpunto2.csv')

# Separar los datos en características (X) y etiquetas (y)
X = dataTrain["titulo"]
y = dataTrain["categoria"]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Inicializar el vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Aplicar el vectorizador a los datos de entrenamiento y prueba
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Inicializar y entrenar el modelo de regresión logística
clf = LogisticRegression(random_state=0, solver='lbfgs', max_iter=100)
clf.fit(X_train_tfidf, y_train)

# Calcular la precisión en el conjunto de entrenamiento y prueba
train_acc = clf.score(X_train_tfidf, y_train)
test_acc = clf.score(X_test_tfidf, y_test)
print('Usando: Regresión logística')
print('Entrenamiento: %.3f, Testeo: %.3f' % (train_acc, test_acc))

Usando: Regresión logística
Entrenamiento: 0.856, Testeo: 0.637


## Random Forest

In [3]:
# Cargar los datos
dataTrain = pd.read_csv('data/train/trainingpunto2.csv')
dataTest = pd.read_csv('data/test/testpunto2.csv')
    
# Separar los datos en características (X) y etiquetas (y)
X = dataTrain["titulo"]
y = dataTrain["categoria"]
    
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
# Inicializar el vectorizador TF-IDF con eliminación de stop words en español
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    
# Aplicar el vectorizador a los datos de entrenamiento y prueba
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
    
# Inicializar y entrenar el modelo Random Forest
rf_classifier = RandomForestClassifier(random_state=0, n_estimators=500)
rf_classifier.fit(X_train_tfidf, y_train)
    
# Calcular la precisión en el conjunto de entrenamiento y prueba
train_acc = rf_classifier.score(X_train_tfidf, y_train)
test_acc = rf_classifier.score(X_test_tfidf, y_test)
print('Usando: Random forest')
print('Entrenamiento: %.3f, Testeo: %.3f' % (train_acc, test_acc))

Usando: Random forest
Entrenamiento: 1.000, Testeo: 0.650


## LSTM (Long short-term memory)

In [6]:
# Cargar los datos
dataTrain = pd.read_csv('data/train/trainingpunto2.csv')
dataTest = pd.read_csv('data/test/testpunto2.csv')

# Separar los datos en características (X) y etiquetas (y)
X = dataTrain["titulo"]
y = dataTrain["categoria"]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar el tokenizador y ajustar el vocabulario
max_words = 25000  # Número máximo de palabras en el vocabulario
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

# Convertir las secuencias de palabras a secuencias de números
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Pad secuencias para que tengan la misma longitud
max_sequence_length = 100  # Longitud máxima de secuencia
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)

# Inicializar un codificador de etiquetas
label_encoder = LabelEncoder()

# Codificar las etiquetas
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Obtener el número de clases después de la codificación
num_classes = len(label_encoder.classes_)

# Crear un modelo LSTM
model = Sequential()
model.add(Embedding(max_words, 64, input_length=max_sequence_length))
model.add(LSTM(64))
model.add(Dense(num_classes, activation='softmax'))  # Capa de salida para clasificación múltiple

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# One-hot encode las etiquetas
y_train_encoded = to_categorical(y_train_encoded, num_classes)
y_test_encoded = to_categorical(y_test_encoded, num_classes)

# Entrenar el modelo
model.fit(X_train_padded, y_train_encoded, epochs=15, batch_size=64)

# Calcular la precisión en el conjunto de entrenamiento y prueba
train_acc = model.evaluate(X_train_padded, y_train_encoded, verbose=0)[1]
test_acc = model.evaluate(X_test_padded, y_test_encoded, verbose=0)[1]
print('Usando: LSTM')
print('Entrenamiento: %.3f, Testeo: %.3f' % (train_acc, test_acc))

Epoch 1/15
10/10 [==============================] - 2s 55ms/step - loss: 1.7649 - accuracy: 0.2906
Epoch 2/15
10/10 [==============================] - 1s 54ms/step - loss: 1.6601 - accuracy: 0.2781
Epoch 3/15
10/10 [==============================] - 1s 55ms/step - loss: 1.6137 - accuracy: 0.3938
Epoch 4/15
10/10 [==============================] - 1s 54ms/step - loss: 1.5492 - accuracy: 0.5141
Epoch 5/15
10/10 [==============================] - 1s 53ms/step - loss: 1.4268 - accuracy: 0.5797
Epoch 6/15
10/10 [==============================] - 1s 53ms/step - loss: 1.3309 - accuracy: 0.5328
Epoch 7/15
10/10 [==============================] - 1s 54ms/step - loss: 1.1875 - accuracy: 0.6516
Epoch 8/15
10/10 [==============================] - 1s 54ms/step - loss: 1.0224 - accuracy: 0.6969
Epoch 9/15
10/10 [==============================] - 1s 55ms/step - loss: 0.8663 - accuracy: 0.7375
Epoch 10/15
10/10 [==============================] - 1s 53ms/step - loss: 0.7361 - accuracy: 0.7750
Epoch 11/

In [5]:
# Obtener los datos de prueba
X_test = dataTest["titulo"]

# Tokenizar y pad las secuencias de prueba
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_sequence_length)

# Realizar predicciones en los datos de prueba
predictions = model.predict(X_test_padded)

# Obtener las categorías predichas para cada instancia
predicted_categories = [label_encoder.classes_[i] for i in predictions.argmax(axis=1)]

# Crear un DataFrame con 'id' y 'categoria' para la sumisión
submission_df = dataTest[['index']]
submission_df['categoria'] = predicted_categories

# Guardar el DataFrame en un archivo CSV
submission_df.to_csv('submission2.csv', index=False)

7/7 [==============================] - 0s 7ms/step


## Matriz de confusion

In [7]:
y_pred_lr = clf.predict(X_test_tfidf)
cm_lr = confusion_matrix(y_test, y_pred_lr)
print("Matriz de Confusión para Regresión Logística:")
print(cm_lr)
report_lr = classification_report(y_test, y_pred_lr)
print(report_lr)

Matriz de Confusión para Regresión Logística:
[[34  0  2  1  0  1]
 [ 4  6  6  0  0  8]
 [ 3  0 38  0  0  7]
 [ 1  0  5  1  0  1]
 [ 2  0  3  0  4  1]
 [ 4  2  7  0  0 19]]
                      precision    recall  f1-score   support

            Deportes       0.71      0.89      0.79        38
           Educación       0.75      0.25      0.38        24
     Entretenimiento       0.62      0.79      0.70        48
       Gente y Blogs       0.50      0.12      0.20         8
               Otros       1.00      0.40      0.57        10
Película y Animación       0.51      0.59      0.55        32

            accuracy                           0.64       160
           macro avg       0.68      0.51      0.53       160
        weighted avg       0.66      0.64      0.61       160



In [8]:
y_pred_rf = rf_classifier.predict(X_test_tfidf)
cm_rf = confusion_matrix(y_test, y_pred_rf)
print("Matriz de Confusión para Random Forest:")
print(cm_rf)
report_rf = classification_report(y_test, y_pred_rf)
print(report_rf)

Matriz de Confusión para Random Forest:
[[31  2  3  1  0  1]
 [ 1 13  6  0  2  2]
 [ 2  2 37  0  0  7]
 [ 1  0  4  3  0  0]
 [ 2  0  3  0  4  1]
 [ 4  6  6  0  0 16]]
                      precision    recall  f1-score   support

            Deportes       0.76      0.82      0.78        38
           Educación       0.57      0.54      0.55        24
     Entretenimiento       0.63      0.77      0.69        48
       Gente y Blogs       0.75      0.38      0.50         8
               Otros       0.67      0.40      0.50        10
Película y Animación       0.59      0.50      0.54        32

            accuracy                           0.65       160
           macro avg       0.66      0.57      0.60       160
        weighted avg       0.65      0.65      0.64       160

